- 筛选出粉丝数目大于大于10w的美妆、美食、游戏、旅行类kol

In [2]:
import numpy as np
import pandas as pd
import jieba
from jieba import posseg
import re
from LAC import LAC

- 读入kol的用户信息

In [3]:
df_kol = pd.read_csv('D:/datasets/qianyi/user_label/file/kollist_gt10w.csv',
                     encoding='utf-8').astype(str)
df_kol

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\software\anaconda\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,kol_id,nick_name,user_type,gender,province,city,vip_level,education,work,birthday,fan_num,follow_num,tweet_num,brief_intro,authentication,label
0,5818685711,Rose宫正,0,f,海外,英国,7,·呼市二中12级,nan,2000-03-01,104026.0,2001.0,5917.0,否极泰来 进一寸有一寸的欢喜,nan,nan
1,5264989659,锦鲤宝宝呀,0,f,上海,nan,7,nan,nan,1997-02-13,102032.0,155.0,1.0,锦鲤宝宝工作室主理人,nan,nan
2,5456865382,摩登兄弟刘宇宁,0,m,辽宁,nan,7,nan,nan,1990-01-08,13379815.0,118.0,1056.0,工作邮箱：mdlynmd@126.com,歌手、演员 超话粉丝大咖（摩登兄弟超话）,nan
3,1878021713,根松仁波且,0,m,西藏,昌都,0,nan,nan,nan,102226.0,804.0,2572.0,http://www.warabuddhism.com/ 瓦拉五明佛学院； 微信公众平台：w...,西藏瓦拉五明佛学院院长,"佛法,菩提,佛教"
4,2606919782,水瓶座-心理学,0,f,江苏,南京,7,·南京大学07级,·中国工商银行股份有限公司·中国石油化工股份有限公司·中国联合网络通信股份有限公司,1989-02-16,5826613.0,194.0,16565.0,解析水瓶座的心理学！~~~~~~~~~ 喜欢就 【✚加关注】｡◕‿◕｡ | 合作QQ：148...,知名情感博主,"星座命理,Aquarius,水瓶女"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200287,6150978462,夜店弹脑门,1,m,nan,nan,6,nan,nan,nan,134002.0,346.0,2650.0,卡我你家割割抠脚10年,婚姻家庭咨询师二级 张妙幸,nan
200288,2609421507,天津乐天百货文化中心店,1,m,nan,nan,0,nan,nan,nan,349396.0,616.0,0.0,nan,天津乐天百货文化中心店官方微博,nan
200289,3877991588,马来西亚千遇旅拍-婚纱摄影-子照,1,f,nan,nan,6,nan,nan,nan,105127.0,291.0,988.0,微信号：elove96，全国热线：400-112-5520官方网站：网站一：http://w...,婚礼博主,nan
200290,6986398702,饭圈安利少年,1,m,nan,nan,4,nan,nan,nan,689126.0,92.0,3065.0,nan,娱乐博主,nan


- 导入分词与停用词函数

In [4]:
def stopwordslist():
    '''
    导入停词表文件
    '''
    \
    stopwords = [line.strip() for line in open('D:/datasets/qianyi/user_label/method/based_on_user_focusing_kol/kol_stop.txt',
                                               encoding='UTF-8').readlines()]
    return stopwords

def seq_jieba(text):  
    '''
    词性标注并筛选（jieba）
    '''
    stopwords = stopwordslist()
    r = jieba.posseg.cut(text)
    a = []
    for w in r:
        #if w.flag in ['n','ns','nr','nz'] and w.word not in stopwords and len(w.word)>1:
        if w.flag in ['n','ns','nr','nz','vn','nz',
                      'nl','s','v','a'] and w.word not in stopwords\
        and len(w.word)>1:
            a.append(w.word)
    return a

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
df_kol['kol_con'] = df_kol['nick_name']+df_kol['brief_intro']+df_kol['label']+df_kol['authentication']
'''将kol的昵称、认证、简介、标签信息整合到一起，后续进行分词'''
df_kol

,kol_id,nick_name,user_type,gender,province,city,vip_level,education,work,birthday,fan_num,follow_num,tweet_num,brief_intro,authentication,label,kol_con
0,5818685711,Rose宫正,0,f,海外,英国,7,·呼市二中12级,nan,2000-03-01,104026.0,2001.0,5917.0,否极泰来 进一寸有一寸的欢喜,nan,nan,Rose宫正否极泰来 进一寸有一寸的欢喜nannan
1,5264989659,锦鲤宝宝呀,0,f,上海,nan,7,nan,nan,1997-02-13,102032.0,155.0,1.0,锦鲤宝宝工作室主理人,nan,nan,锦鲤宝宝呀锦鲤宝宝工作室主理人nannan
2,5456865382,摩登兄弟刘宇宁,0,m,辽宁,nan,7,nan,nan,1990-01-08,13379815.0,118.0,1056.0,工作邮箱：mdlynmd@126.com,歌手、演员 超话粉丝大咖（摩登兄弟超话）,nan,摩登兄弟刘宇宁工作邮箱：mdlynmd@126.comnan歌手、演员 超话粉丝大咖（摩登兄...
3,1878021713,根松仁波且,0,m,西藏,昌都,0,nan,nan,nan,102226.0,804.0,2572.0,http://www.warabuddhism.com/ 瓦拉五明佛学院； 微信公众平台：w...,西藏瓦拉五明佛学院院长,"佛法,菩提,佛教",根松仁波且http://www.warabuddhism.com/ 瓦拉五明佛学院； 微信公...
4,2606919782,水瓶座-心理学,0,f,江苏,南京,7,·南京大学07级,·中国工商银行股份有限公司·中国石油化工股份有限公司·中国联合网络通信股份有限公司,1989-02-16,5826613.0,194.0,16565.0,解析水瓶座的心理学！~~~~~~~~~ 喜欢就 【✚加关注】｡◕‿◕｡ | 合作QQ：148...,知名情感博主,"星座命理,Aquarius,水瓶女",水瓶座-心理学解析水瓶座的心理学！~~~~~~~~~ 喜欢就 【✚加关注】｡◕‿◕｡ | 合...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200287,6150978462,夜店弹脑门,1,m,nan,nan,6,nan,nan,nan,134002.0,346.0,2650.0,卡我你家割割抠脚10年,婚姻家庭咨询师二级 张妙幸,nan,夜店弹脑门卡我你家割割抠脚10年nan婚姻家庭咨询师二级 张妙幸
200288,2609421507,天津乐天百货文化中心店,1,m,nan,nan,0,nan,nan,nan,349396.0,616.0,0.0,nan,天津乐天百货文化中心店官方微博,nan,天津乐天百货文化中心店nannan天津乐天百货文化中心店官方微博
200289,3877991588,马来西亚千遇旅拍-婚纱摄影-子照,1,f,nan,nan,6,nan,nan,nan,105127.0,291.0,988.0,微信号：elove96，全国热线：400-112-5520官方网站：网站一：http://w...,婚礼博主,nan,马来西亚千遇旅拍-婚纱摄影-子照微信号：elove96，全国热线：400-112-5520官...
200290,6986398702,饭圈安利少年,1,m,nan,nan,4,nan,nan,nan,689126.0,92.0,3065.0,nan,娱乐博主,nan,饭圈安利少年nannan娱乐博主


- 分词

In [7]:
import time
s_t = time.time()

con_seq = df_kol.kol_con.apply(lambda x: seq_jieba(x))

print(time.time()-s_t)

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LEOZHO~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.288 seconds.
Prefix dict has been built successfully.


925.7096393108368


In [9]:
df_kol['con_seq'] = con_seq
df_kol

,kol_id,nick_name,user_type,gender,province,city,vip_level,education,work,birthday,fan_num,follow_num,tweet_num,brief_intro,authentication,label,kol_con,con_seq
0,5818685711,Rose宫正,0,f,海外,英国,7,·呼市二中12级,nan,2000-03-01,104026.0,2001.0,5917.0,否极泰来 进一寸有一寸的欢喜,nan,nan,Rose宫正否极泰来 进一寸有一寸的欢喜nannan,"[宫正, 欢喜]"
1,5264989659,锦鲤宝宝呀,0,f,上海,nan,7,nan,nan,1997-02-13,102032.0,155.0,1.0,锦鲤宝宝工作室主理人,nan,nan,锦鲤宝宝呀锦鲤宝宝工作室主理人nannan,"[锦鲤, 宝宝, 锦鲤, 宝宝, 工作室, 主理]"
2,5456865382,摩登兄弟刘宇宁,0,m,辽宁,nan,7,nan,nan,1990-01-08,13379815.0,118.0,1056.0,工作邮箱：mdlynmd@126.com,歌手、演员 超话粉丝大咖（摩登兄弟超话）,nan,摩登兄弟刘宇宁工作邮箱：mdlynmd@126.comnan歌手、演员 超话粉丝大咖（摩登兄...,"[兄弟, 刘宇宁, 邮箱, 歌手, 演员, 超话, 粉丝, 兄弟, 超话]"
3,1878021713,根松仁波且,0,m,西藏,昌都,0,nan,nan,nan,102226.0,804.0,2572.0,http://www.warabuddhism.com/ 瓦拉五明佛学院； 微信公众平台：w...,西藏瓦拉五明佛学院院长,"佛法,菩提,佛教",根松仁波且http://www.warabuddhism.com/ 瓦拉五明佛学院； 微信公...,"[松仁, 五明, 佛学院, 微信, 公众, 平台, 佛法, 菩提, 佛教, 西藏, 五明, ..."
4,2606919782,水瓶座-心理学,0,f,江苏,南京,7,·南京大学07级,·中国工商银行股份有限公司·中国石油化工股份有限公司·中国联合网络通信股份有限公司,1989-02-16,5826613.0,194.0,16565.0,解析水瓶座的心理学！~~~~~~~~~ 喜欢就 【✚加关注】｡◕‿◕｡ | 合作QQ：148...,知名情感博主,"星座命理,Aquarius,水瓶女",水瓶座-心理学解析水瓶座的心理学！~~~~~~~~~ 喜欢就 【✚加关注】｡◕‿◕｡ | 合...,"[水瓶座, 心理学, 解析, 水瓶座, 心理学, 喜欢, 合作, 星座, 命理, 水瓶, 知..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200287,6150978462,夜店弹脑门,1,m,nan,nan,6,nan,nan,nan,134002.0,346.0,2650.0,卡我你家割割抠脚10年,婚姻家庭咨询师二级 张妙幸,nan,夜店弹脑门卡我你家割割抠脚10年nan婚姻家庭咨询师二级 张妙幸,"[夜店, 脑门, 割割, 婚姻家庭, 咨询师, 张妙幸]"
200288,2609421507,天津乐天百货文化中心店,1,m,nan,nan,0,nan,nan,nan,349396.0,616.0,0.0,nan,天津乐天百货文化中心店官方微博,nan,天津乐天百货文化中心店nannan天津乐天百货文化中心店官方微博,"[天津, 乐天, 百货, 文化, 中心店, 天津, 乐天, 百货, 文化, 中心店, 官方]"
200289,3877991588,马来西亚千遇旅拍-婚纱摄影-子照,1,f,nan,nan,6,nan,nan,nan,105127.0,291.0,988.0,微信号：elove96，全国热线：400-112-5520官方网站：网站一：http://w...,婚礼博主,nan,马来西亚千遇旅拍-婚纱摄影-子照微信号：elove96，全国热线：400-112-5520官...,"[马来西亚, 婚纱, 摄影, 子照微, 全国, 热线, 官方网站, 网站, 网站, 婚礼, 博主]"
200290,6986398702,饭圈安利少年,1,m,nan,nan,4,nan,nan,nan,689126.0,92.0,3065.0,nan,娱乐博主,nan,饭圈安利少年nannan娱乐博主,"[饭圈, 安利, 娱乐, 博主]"


- kol分词文件 输出

In [10]:
df_kol.to_csv('D:/datasets/qianyi/user_label/method/based_on_user_focusing_kol/file/kol_seq_result.csv',index=False)

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<font color=red>4万kol整合信息分词耗时9min</font>

<font color=red>20万kol整合信息分词耗时36min（15min）</font>

### 筛选出美妆类的博主

In [6]:
make_up_list = ['美妆','化妆','彩妆','护肤','美容']         #与美妆相关的词语

- 美妆类博主id

In [7]:
make_up_index = [j for i in range(len(make_up_list)) for j \
                 in range(len(df_kol)) if make_up_list[i] in con_seq[j]] #词库信息与kol分词结果进行匹配
'''
寻找美妆类博主的id
'''
print(make_up_index)
print(len(make_up_index))

[150, 216, 230, 985, 1044, 1457, 1472, 1516, 1602, 2717, 2886, 3893, 4692, 4811, 4857, 4890, 5275, 5737, 5990, 6510, 6923, 7076, 7650, 7963, 8046, 8073, 8550, 8581, 8867, 8882, 9542, 9966, 9974, 11154, 11903, 13208, 13818, 13910, 14008, 14135, 14516, 14982, 15802, 16293, 17159, 17473, 17652, 17852, 17873, 18595, 19043, 19064, 20307, 20605, 21223, 21489, 21526, 22803, 23690, 24027, 24091, 24109, 25095, 25310, 25571, 25652, 26405, 26575, 26673, 26914, 27805, 28546, 28549, 28665, 29309, 29344, 29459, 29883, 30611, 30624, 31399, 31487, 33006, 33384, 33908, 34051, 34097, 34914, 35525, 36130, 36194, 37425, 37489, 39287, 39623, 40161, 40169, 40718, 41732, 41781, 42444, 42688, 42837, 43507, 43803, 44407, 44469, 45361, 45589, 46382, 46421, 46579, 46583, 46782, 47254, 47308, 47907, 47986, 48234, 48383, 48429, 49794, 50210, 51269, 53110, 53935, 53998, 54365, 54906, 54968, 55094, 55837, 56075, 56762, 57359, 57382, 57451, 57605, 57703, 57912, 58267, 59049, 59692, 60017, 60314, 60494, 61083, 61108, 

In [8]:
df_makeup = df_kol.iloc[make_up_index]
df_makeup

,kol_id,nick_name,user_type,gender,province,city,vip_level,education,work,birthday,fan_num,follow_num,tweet_num,brief_intro,authentication,label,kol_con
150,6111217365,你柚有点甜儿,1,f,上海,nan,7,nan,nan,1995-08-31,2012320.0,162.0,1174.0,美妆 时尚 穿搭！,知名美妆博主 微博原创视频博主 微博VLOG博主,"吃喝玩乐,旅游摄影,化妆品","你柚有点甜儿美妆 时尚 穿搭！吃喝玩乐,旅游摄影,化妆品知名美妆博主 微博原创视频博主 微博..."
216,7294063713,六六酱llj,1,f,上海,nan,4,nan,nan,1995-01-01,597497.0,45.0,159.0,专研美妆的资深空乘，有任何美妆方面的问题都可以帮你解决～合作VX:xuyan388（备注来意...,美妆博主 微博原创视频博主,nan,六六酱llj专研美妆的资深空乘，有任何美妆方面的问题都可以帮你解决～合作VX:xuyan38...
230,7457579680,美妆挖掘机_,1,f,上海,nan,4,nan,nan,1996-05-19,1143680.0,76.0,196.0,只有你想不到的美妆没有我挖不倒 合作加🛰caijia85,美妆博主,"美妆护肤,美妆控","美妆挖掘机_只有你想不到的美妆没有我挖不倒 合作加🛰caijia85美妆护肤,美妆控美妆博主"
985,6358577421,仙女也爱买,1,f,江苏,南京,2,·北京邮电大学09级,·徐州微达文化传播有限公司2016年-2017年,1993-11-28,141203.0,95.0,128824.0,专注种草拔草的美妆博~,时尚博主,"时尚,护肤,美妆","仙女也爱买专注种草拔草的美妆博~时尚,护肤,美妆时尚博主"
1044,1564595531,-元气六六,1,f,广东,nan,7,nan,·么么哒1998年-2015年,01-01,391819.0,57.0,30109.0,喜欢买喜欢分享的美妆博主。,美妆博主,"口红,化妆,护肤","-元气六六喜欢买喜欢分享的美妆博主。口红,化妆,护肤美妆博主"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199845,2402865684,Angel芭比成都权威整形修复,2,f,nan,nan,6,nan,nan,nan,101018.0,189.0,554.0,成都整形美容协会，中国整形协会认证指定修复基地。,nan,nan,Angel芭比成都权威整形修复成都整形美容协会，中国整形协会认证指定修复基地。nannan
199863,3964927653,卡通保温杯,1,f,nan,nan,0,nan,nan,nan,150807.0,1518.0,2504.0,B365水果酵素养生、美容！想要了解更多的加我微信吧😄yyz20121015,富阳雅高机械维修经营部 会计 李珊,nan,卡通保温杯B365水果酵素养生、美容！想要了解更多的加我微信吧😄yyz20121015nan...
199970,3911142496,皮肤管理院线美容仪器批发货源,2,f,nan,nan,4,nan,nan,nan,114725.0,1723.0,1810.0,皮肤管理美容仪器厂家一手货源💕184 1266 003,nan,nan,皮肤管理院线美容仪器批发货源皮肤管理美容仪器厂家一手货源💕184 1266 003nannan
199973,1631766365,丁子Queena,1,f,nan,nan,5,nan,nan,nan,198688.0,222.0,458.0,科颜美皮肤管理中心创始人 时尚造型师 知名护肤专家，品牌特邀嘉宾，合作请私信,致美焕颜美容会所创始人,nan,丁子Queena科颜美皮肤管理中心创始人 时尚造型师 知名护肤专家，品牌特邀嘉宾，合作请私信...


In [9]:
df_makeup.to_csv('D:/datasets/qianyi/user_label/method/based_on_user_focusing_kol/file/makeup_kol_gt10w.csv',index=False)

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 筛选出美食类kol

In [10]:
food_index = [j for j in range(len(df_kol)) if '美食' in con_seq[j]]
'''
寻找美妆类博主的id
'''
print(food_index)
print(len(food_index))

[5, 8, 27, 40, 44, 59, 74, 93, 119, 141, 153, 156, 170, 171, 174, 196, 199, 251, 257, 274, 282, 291, 323, 333, 360, 369, 370, 398, 404, 407, 452, 461, 467, 470, 474, 479, 483, 501, 524, 537, 600, 610, 615, 620, 626, 635, 668, 677, 687, 706, 719, 724, 730, 735, 739, 752, 768, 803, 804, 821, 834, 897, 906, 907, 908, 909, 948, 955, 973, 986, 998, 1025, 1032, 1042, 1052, 1058, 1073, 1084, 1086, 1093, 1101, 1113, 1119, 1160, 1188, 1197, 1208, 1217, 1224, 1248, 1254, 1283, 1299, 1300, 1310, 1319, 1344, 1353, 1367, 1374, 1399, 1431, 1433, 1440, 1446, 1467, 1500, 1513, 1523, 1527, 1528, 1535, 1552, 1556, 1585, 1593, 1641, 1665, 1684, 1700, 1781, 1800, 1814, 1840, 1842, 1852, 1892, 1904, 1911, 1927, 1950, 1963, 1967, 1973, 1980, 1983, 1985, 1992, 1996, 2004, 2064, 2078, 2096, 2099, 2108, 2146, 2147, 2152, 2156, 2167, 2171, 2173, 2177, 2191, 2230, 2238, 2258, 2279, 2282, 2283, 2287, 2291, 2305, 2307, 2316, 2321, 2326, 2372, 2374, 2396, 2398, 2406, 2415, 2420, 2426, 2429, 2436, 2492, 2498, 2499, 

In [11]:
df_food = df_kol.iloc[food_index]
df_food

,kol_id,nick_name,user_type,gender,province,city,vip_level,education,work,birthday,fan_num,follow_num,tweet_num,brief_intro,authentication,label,kol_con
5,5899491342,美食零食小吃货,0,m,安徽,合肥,6,·汉口学院98级,·情感公司2003年-2020年,2011-11-26,18920185.0,583.0,3969.0,商务合作：ok311331,作家 情感博主 微博原创视频博主,nan,美食零食小吃货商务合作：ok311331nan作家 情感博主 微博原创视频博主
8,5889860264,郭诗妤Amarisbaby,0,f,上海,nan,7,·厦门大学,nan,1998-09-16,236727.0,148.0,137.0,淘宝：Amarisbaby. 店铺客服V:shiyu916111（客服3号美妞加） 工作...,nan,"美食,美女,美容护肤",郭诗妤Amarisbaby淘宝：Amarisbaby. 店铺客服V:shiyu91611...
27,1606653640,Cream_Paul甜点教学,0,m,北京,nan,7,nan,·CREAM PAUL 甜品教学2011年-·北京国贸中心国贸三期酒店2010年-2011年...,nan,192489.0,2801.0,3209.0,法国里昂西点世界杯中国赛区选拔赛冠军成员我叫PAUL 老婆叫CREAM 服务于有理想追求的甜...,美食博主 美食视频自媒体,"巧克力,拉糖,甜品教学",Cream_Paul甜点教学法国里昂西点世界杯中国赛区选拔赛冠军成员我叫PAUL 老婆叫CR...
40,6330267311,凌晨三点半来了,0,m,黑龙江,哈尔滨,2,nan,nan,1994-07-11,105636.0,418.0,408.0,你来，我就欢迎。,微博新知博主 互联网科技博主,"吃货,美食,篮球","凌晨三点半来了你来，我就欢迎。吃货,美食,篮球微博新知博主 互联网科技博主"
44,1302496952,如昨,0,m,福建,福州,7,nan,nan,01-01,265595.0,1374.0,8936.0,爱，就关注吧。即投缘，又缘何擦肩而过....对于福州历史典故，民俗、文化、方言、俚语、建筑、...,微博美食博主 超话粉丝大咖（投闽状超话）,"股票,登山看海,听歌",如昨爱，就关注吧。即投缘，又缘何擦肩而过....对于福州历史典故，民俗、文化、方言、俚语、建...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200025,6582920476,前线小面包,1,f,nan,nan,6,nan,nan,nan,127660.0,509.0,1682.0,陪lmy走过漫长岁月.喜欢美食、旅行、读书🐰🐰,娱乐博主,nan,前线小面包陪lmy走过漫长岁月.喜欢美食、旅行、读书🐰🐰nan娱乐博主
200103,6506168038,北京中诺口腔医院-种植中心,1,m,nan,nan,1,nan,nan,nan,301255.0,431.0,408.0,北京中诺口腔医院，北京口腔医院、第四军医大专家团队，采用立得用种植牙体系，当天来院，当天种牙...,三级口腔医院，立得用数字化种植牙体系，当天种牙当天享美食！,nan,北京中诺口腔医院-种植中心北京中诺口腔医院，北京口腔医院、第四军医大专家团队，采用立得用种植...
200156,3225618982,万达大歌星KTV长沙店,1,f,nan,nan,0,nan,nan,nan,109373.0,1660.0,6409.0,大歌星KTV创立于2010年6月，是万达集团旗下的大型文化连锁企业，是集休闲娱乐、欢唱聚会、...,万达大歌星KTV长沙店,nan,万达大歌星KTV长沙店大歌星KTV创立于2010年6月，是万达集团旗下的大型文化连锁企业，是...
200229,2378430134,伯爵先苼的家常美味,1,m,nan,nan,6,nan,nan,nan,151720.0,233.0,2998.0,吃在路上ing、终身阅读者,携程社区签约旅行家 美食博主,nan,伯爵先苼的家常美味吃在路上ing、终身阅读者nan携程社区签约旅行家 美食博主


In [12]:
df_food.to_csv('D:/datasets/qianyi/user_label/method/based_on_user_focusing_kol/file/food_kol_gt10w.csv',index=False)

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 筛选出旅行类kol 

In [13]:
travel_list = ['旅行','旅游']         #与美妆相关的词语

In [14]:
travel_index = [j for i in range(len(travel_list)) for\
                j in range(len(df_kol)) if travel_list[i] in con_seq[j]]
'''
寻找美妆类博主的id
'''
print(travel_index)
print(len(travel_index))

[7, 47, 63, 76, 100, 123, 129, 153, 179, 183, 187, 196, 199, 222, 263, 272, 359, 436, 472, 474, 527, 610, 620, 701, 713, 733, 815, 826, 922, 1015, 1062, 1075, 1167, 1241, 1251, 1257, 1265, 1283, 1310, 1355, 1362, 1404, 1440, 1445, 1456, 1491, 1501, 1504, 1514, 1517, 1523, 1555, 1589, 1653, 1670, 1723, 1775, 1806, 1847, 1911, 1930, 1940, 1957, 2064, 2083, 2086, 2123, 2126, 2146, 2175, 2209, 2218, 2280, 2326, 2373, 2381, 2390, 2398, 2419, 2442, 2482, 2583, 2615, 2643, 2678, 2775, 2785, 2803, 2831, 2909, 2950, 2999, 3004, 3009, 3068, 3082, 3275, 3280, 3335, 3391, 3414, 3458, 3589, 3600, 3641, 3697, 3701, 3706, 3720, 3774, 3849, 3897, 3942, 3946, 3953, 3963, 3981, 4053, 4066, 4087, 4132, 4139, 4199, 4228, 4299, 4333, 4362, 4439, 4475, 4505, 4525, 4576, 4670, 4685, 4714, 4848, 4853, 4870, 4885, 4949, 4971, 4993, 4995, 4996, 5020, 5023, 5024, 5039, 5048, 5090, 5103, 5108, 5145, 5154, 5167, 5254, 5264, 5266, 5271, 5276, 5290, 5305, 5336, 5345, 5347, 5440, 5458, 5463, 5470, 5544, 5563, 5568, 5

In [15]:
df_travel = df_kol.iloc[travel_index]
df_travel

,kol_id,nick_name,user_type,gender,province,city,vip_level,education,work,birthday,fan_num,follow_num,tweet_num,brief_intro,authentication,label,kol_con
7,3263402975,我是CHLOE,0,f,上海,nan,6,·华东理工大学,nan,2000-03-01,228408.0,458.0,577.0,旅行玩家，去过五十几个国家,nan,"穿衣美容,投资理财","我是CHLOE旅行玩家，去过五十几个国家穿衣美容,投资理财nan"
47,5014296291,婀娜多姿拉拉,0,f,陕西,nan,7,nan,nan,nan,1048394.0,7753.0,32482.0,旅行，只有起点，没有终点。,知名旅游博主 去哪儿网聪明旅行家 超话粉丝大咖（小脚丫走世界超话） 微博故事原创作者 旅行视...,nan,婀娜多姿拉拉旅行，只有起点，没有终点。nan知名旅游博主 去哪儿网聪明旅行家 超话粉丝大咖（...
63,5606936822,自然卷_卷卷,0,f,新疆,乌鲁木齐,4,·陕西中医药大学10级,nan,1996-01-02,218714.0,101.0,161.0,还在努力的一名混干皮 看似温柔实则沙雕 喜欢摄影 热爱旅行 日常种草：护肤｜美妆｜生活好物...,旅游博主 微博原创视频博主,"摄影控,护肤,喜欢旅游",自然卷_卷卷还在努力的一名混干皮 看似温柔实则沙雕 喜欢摄影 热爱旅行 日常种草：护肤｜美...
76,2358288921,小小攻略菌,0,f,内蒙古,nan,6,·澳门旅游学院11级·澳门旅游学院04级·澳门旅游学院,·九零后旅行社2009年-2015年,1992-07-06,446322.0,2044.0,2177.0,旅拍中国，打卡小众景点，享受自由行带来的舒心旅行！,旅游博主 马蜂窝攻略作者,"成都生活,旅遊,爱自由","小小攻略菌旅拍中国，打卡小众景点，享受自由行带来的舒心旅行！成都生活,旅遊,爱自由旅游博主 ..."
100,1686128307,紫荆__,1,f,天津,河西区,6,nan,nan,nan,1247566.0,628.0,24669.0,专栏作家、旅行玩家、美妆达人；weix：lljing2010,时尚达人 乐途旅游网特约旅行家 旅游体验师 同程旅行推荐官 知名旅游博主 头条文章作者,"自驾,旅游,嘉人美妆精英","紫荆__专栏作家、旅行玩家、美妆达人；weix：lljing2010自驾,旅游,嘉人美妆精英..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199952,5915608851,陕西逗姐,1,f,nan,nan,4,nan,nan,nan,205188.0,2181.0,2709.0,爱生活 爱旅游,旅游博主,nan,陕西逗姐爱生活 爱旅游nan旅游博主
199960,5533452867,彩云天上飘,1,f,nan,nan,6,nan,nan,nan,152836.0,5033.0,12406.0,喜欢在旅行中带来的快乐～,旅游博主,nan,彩云天上飘喜欢在旅行中带来的快乐～nan旅游博主
199989,5393185738,酷酷滴小峰哥,1,m,nan,nan,1,nan,nan,nan,128459.0,726.0,706.0,nan,旅游博主,nan,酷酷滴小峰哥nannan旅游博主
200106,1426709441,I赤龙飞天I,2,m,nan,nan,6,nan,nan,nan,143783.0,2005.0,2736.0,爱旅游 爱摄影 爱交天下朋友；,nan,nan,I赤龙飞天I爱旅游 爱摄影 爱交天下朋友；nannan


In [16]:
df_travel.to_csv('D:/datasets/qianyi/user_label/method/based_on_user_focusing_kol/file/travel_kol_gt10w.csv',index=False)

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 筛选出游戏类kol

In [17]:
game_list = ['游戏','手游']         #与美妆相关的词语

In [18]:
game_index = [j for i in range(len(game_list)) for\
                j in range(len(df_kol)) if game_list[i] in con_seq[j]]
'''
寻找美妆类博主的id
'''
print(game_index)
print(len(game_index))

[119, 135, 186, 234, 245, 328, 448, 515, 617, 631, 634, 650, 688, 750, 758, 794, 822, 860, 903, 943, 947, 1019, 1110, 1163, 1217, 1325, 1328, 1370, 1383, 1389, 1471, 1618, 1703, 1750, 1758, 1841, 1861, 1868, 1869, 1897, 1898, 1918, 1946, 2046, 2121, 2139, 2165, 2199, 2236, 2264, 2288, 2322, 2347, 2351, 2357, 2379, 2451, 2461, 2466, 2538, 2567, 2571, 2597, 2753, 2844, 2867, 2976, 2983, 3012, 3022, 3043, 3049, 3106, 3113, 3115, 3181, 3208, 3296, 3334, 3382, 3421, 3448, 3464, 3748, 3813, 3824, 3829, 3854, 3855, 3857, 3867, 3902, 4012, 4023, 4060, 4158, 4220, 4229, 4244, 4248, 4259, 4409, 4432, 4459, 4623, 4638, 4641, 4644, 4653, 4718, 4728, 4735, 4780, 4886, 4981, 4987, 5026, 5052, 5132, 5136, 5246, 5287, 5319, 5368, 5412, 5502, 5571, 5640, 5665, 5691, 5702, 5815, 5818, 5860, 5922, 5970, 5992, 6010, 6057, 6099, 6193, 6286, 6338, 6389, 6396, 6403, 6430, 6503, 6521, 6547, 6554, 6587, 6597, 6690, 6699, 6755, 6826, 6835, 6896, 6912, 6921, 6972, 6981, 7007, 7071, 7079, 7162, 7169, 7259, 7276, 

In [19]:
df_game = df_kol.iloc[game_index]
df_game

,kol_id,nick_name,user_type,gender,province,city,vip_level,education,work,birthday,fan_num,follow_num,tweet_num,brief_intro,authentication,label,kol_con
119,1271018062,星语美食娱乐,2,f,海外,美国,0,·信达外国语学校,nan,nan,104193.0,91.0,5125.0,Q:326888528,nan,"星座命理,游戏,美剧","星语美食娱乐Q:326888528星座命理,游戏,美剧nan"
135,2097210323,正宗好游戏,2,f,四川,成都,0,nan,nan,01-01,1031574.0,498.0,7091.0,最新最全的游戏综合资讯~尽在@正宗好游戏,nan,"极品飞车,实况2014,游戏爱好者","正宗好游戏最新最全的游戏综合资讯~尽在@正宗好游戏极品飞车,实况2014,游戏爱好者nan"
186,1233143520,微为烟夕罗,1,f,上海,闸北区,7,nan,nan,nan,152037.0,747.0,70060.0,叫我萝卜就口以 淘宝店铺：烟夕罗捏脸铺子,游戏博主,"微博刷屏王,宅,要命的处女座","微为烟夕罗叫我萝卜就口以 淘宝店铺：烟夕罗捏脸铺子微博刷屏王,宅,要命的处女座游戏博主"
234,5360483141,Ow_Diya,1,m,上海,nan,7,nan,·上海龙之队,1999-07-30,135387.0,345.0,868.0,就是人帅了一点游戏也打得好没办法的。,Shanghai Dragons上海龙之队成员,"游戏,vgdiya,电子竞技","Ow_Diya就是人帅了一点游戏也打得好没办法的。游戏,vgdiya,电子竞技Shangha..."
245,3938580330,极客羊,1,m,广东,广州,6,nan,·极客羊工作室（Geek Sheep Studio）2018年-至今,2018-04-13,106198.0,11.0,253.0,“万物皆可盘彩蛋，世事均能抠细节” 坚持用心做好每一个彩蛋视频～ 每周二周五更新～ (≧▽≦...,游戏博主,"彩蛋,游戏,软件",极客羊“万物皆可盘彩蛋，世事均能抠细节” 坚持用心做好每一个彩蛋视频～ 每周二周五更新～ (...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195638,5035554105,PopStar消灭星星,1,f,北京,海淀区,0,nan,nan,1979-07-01,586657.0,360.0,2096.0,《PopStar!消灭星星官方正版》是最经典的消除类休闲手游，全球拥有超过4亿用户，始终是休...,PopStar消灭星星官方微博,"爱猫,八卦,好玩的",PopStar消灭星星《PopStar!消灭星星官方正版》是最经典的消除类休闲手游，全球拥有...
195762,5556747367,手游主播君,1,f,海外,nan,0,nan,nan,nan,1138509.0,150.0,50.0,知名游戏博主,游戏博主,"名人明星,新闻资讯,游戏动漫","手游主播君知名游戏博主名人明星,新闻资讯,游戏动漫游戏博主"
196144,3985622865,剑魂之刃官博,1,f,北京,海淀区,0,nan,·北京昆仑乐享网络技术有限公司,0001-00-00,1141378.0,55.0,108.0,全球首款手势操控的格斗手游《剑魂之刃》，备受国民瞩目的IOS版8.13日正式火爆上线！超值大...,剑魂之刃官方微博,"手势格斗,竞技,拳皇",剑魂之刃官博全球首款手势操控的格斗手游《剑魂之刃》，备受国民瞩目的IOS版8.13日正式火爆...
198444,3338517620,NBA梦之队官博,1,m,nan,nan,0,nan,nan,nan,1066395.0,102.0,8372.0,NBA官方授权的正版篮球社交手游 《NBA梦之队》，官网nba.mobage.cn/home,NBA梦之队官方微博,nan,NBA梦之队官博NBA官方授权的正版篮球社交手游 《NBA梦之队》，官网nba.mobage...


In [20]:
df_game.to_csv('D:/datasets/qianyi/user_label/method/based_on_user_focusing_kol/file/game_kol_gt10w.csv',index=False)

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 筛选出搞笑幽默类kol 

In [12]:
fun_list = ['搞笑','幽默']                                                     #与搞笑相关的词语
fun_index = [j for i in range(len(fun_list)) for\
                j in range(len(df_kol)) if fun_list[i] in con_seq[j]]          #搞笑类博主ID

In [13]:
print(fun_index)
print(len(fun_index))

df_fun = df_kol.iloc[fun_index]
df_fun

[30, 57, 64, 66, 103, 113, 130, 185, 187, 200, 223, 243, 268, 270, 277, 291, 323, 325, 360, 362, 371, 377, 511, 519, 530, 540, 576, 617, 618, 651, 668, 675, 714, 732, 733, 739, 751, 754, 758, 760, 761, 787, 833, 862, 927, 951, 966, 1026, 1033, 1034, 1061, 1066, 1078, 1086, 1087, 1099, 1122, 1136, 1155, 1182, 1209, 1229, 1230, 1255, 1304, 1314, 1328, 1359, 1368, 1389, 1394, 1404, 1429, 1458, 1466, 1478, 1479, 1498, 1503, 1540, 1574, 1582, 1598, 1600, 1601, 1628, 1645, 1646, 1660, 1694, 1696, 1780, 1781, 1784, 1795, 1798, 1803, 1808, 1848, 1874, 1962, 1968, 1969, 2007, 2070, 2072, 2091, 2108, 2173, 2190, 2216, 2222, 2245, 2246, 2257, 2262, 2273, 2276, 2300, 2311, 2316, 2322, 2407, 2425, 2443, 2459, 2541, 2553, 2585, 2645, 2721, 2726, 2743, 2753, 2755, 2758, 2779, 2789, 2820, 2859, 2922, 2939, 2953, 2983, 3022, 3026, 3028, 3045, 3106, 3151, 3185, 3203, 3257, 3270, 3304, 3313, 3319, 3403, 3409, 3427, 3452, 3454, 3521, 3547, 3584, 3591, 3623, 3625, 3638, 3648, 3652, 3763, 3766, 3775, 3807, 

,kol_id,nick_name,user_type,gender,province,city,vip_level,education,work,birthday,fan_num,follow_num,tweet_num,brief_intro,authentication,label,kol_con,con_seq
30,2301726604,卓宣zoey,0,f,北京,nan,7,nan,nan,天秤座,2870800.0,394.0,1510.0,没有人能定义我（合作加v：jacm188）,美妆博主,"生活,搞笑视频,搞笑图片","卓宣zoey没有人能定义我（合作加v：jacm188）生活,搞笑视频,搞笑图片美妆博主","[卓宣, 人能, 定义, 合作, 生活, 搞笑, 视频, 搞笑, 图片, 博主]"
57,3015928115,国人养生殿孝基,0,m,北京,西城区,1,nan,nan,nan,104664.0,2195.0,1110.0,XXXWL9527❤V,搞笑幽默博主,美女,国人养生殿孝基XXXWL9527❤V美女搞笑幽默博主,"[国人, 养生, 美女, 搞笑, 幽默, 博主]"
64,5025857653,果子,0,m,河南,郑州,6,nan,nan,狮子座,20166337.0,234.0,99327.0,商务合作v：55188955,搞笑幽默博主 微博原创视频博主 微博VLOG博主,nan,果子商务合作v：55188955nan搞笑幽默博主 微博原创视频博主 微博VLOG博主,"[果子, 商务, 合作, 搞笑, 幽默, 博主, 原创, 视频, 博主, 博主]"
66,5362566664,著名造型师peter,0,m,福建,nan,7,nan,·MAXhairsalon2015年-2016年·昕薇杂志,1980-03-11,170455.0,319.0,1618.0,nan,时尚博主,搞笑幽默,著名造型师peternan搞笑幽默时尚博主,"[著名, 造型师, 搞笑, 幽默, 时尚, 博主]"
103,6811589462,喝大了,1,m,浙江,nan,5,·浙江大学14级,·,1996-05-17,2026534.0,29.0,5774.0,关注我的听说都找到配对了。,搞笑幽默博主,nan,喝大了关注我的听说都找到配对了。nan搞笑幽默博主,"[听说, 找到, 配对, 搞笑, 幽默, 博主]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199532,2419121401,就笑了怎么,1,m,nan,nan,6,nan,nan,nan,275877.0,500.0,695.0,哈哈哈哈，不想再被硬邦邦的现实撞得人仰马翻,搞笑幽默博主,nan,就笑了怎么哈哈哈哈，不想再被硬邦邦的现实撞得人仰马翻nan搞笑幽默博主,"[不想, 现实, 搞笑, 幽默, 博主]"
199761,1974750773,幽默咖1,2,m,nan,nan,2,nan,nan,nan,122913.0,1921.0,225.0,🔥世界顶尖搞笑大咖cc,nan,nan,幽默咖1🔥世界顶尖搞笑大咖ccnannan,"[幽默, 顶尖, 搞笑]"
199829,1788096093,银河系小喷子,1,m,nan,nan,6,nan,nan,nan,126427.0,994.0,1876.0,说那么多都是为了什么呢？,搞笑幽默博主,nan,银河系小喷子说那么多都是为了什么呢？nan搞笑幽默博主,"[银河系, 喷子, 搞笑, 幽默, 博主]"
199852,3889228788,欢乐的Jony格轮,1,f,nan,nan,0,nan,nan,nan,100911.0,2870.0,2129.0,只有经历过地狱般的折磨，才有征服天堂的力量。只有流过血的手指才能弹出世间的绝唱。,搞笑幽默博主,nan,欢乐的Jony格轮只有经历过地狱般的折磨，才有征服天堂的力量。只有流过血的手指才能弹出世间的...,"[欢乐, 格轮, 地狱, 折磨, 征服, 天堂, 力量, 流过, 手指, 才能, 世间, 绝..."


In [14]:
df_fun.to_csv('D:/datasets/qianyi/user_label/method/based_on_user_focusing_kol/file/kol/fun_kol_gt10w.csv',index=False)

D:\software\anaconda\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
